In [ ]:
import numpy as np
import pyvisa
import time
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from scipy.optimize import minimize
from scipy.stats import norm
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# ==========================================
# 1. PHYSICAL CONSTANTS & CONFIGURATION ⚛️
# ==========================================
class Config:
    # Physical Constants
    e = 1.60217663e-19
    f = 0.350 * 1e9  # Pump frequency in Hz (Adjust to your experiment!)
    target_current = e * f
    
    # GPIB Addresses (Update based on your rack)
    addr_yoko_ent = "GPIB0::11::INSTR"  # G_ENT
    addr_yoko_p   = "GPIB0::7::INSTR"   # G_P (Assumed new source)
    addr_yoko_exit= "GPIB0::8::INSTR"   # G_EXIT (Assumed new source)
    addr_dmm      = "GPIB0::22::INSTR"  # Multimeter
    
    # Optimization Constraints (Safety Bounds in Volts)
    # Order: [V_ENT, V_P, V_EXIT]
    bounds = np.array([
        [-0.8, -0.4], # V_ENT range
        [-0.8, -0.4], # V_P range
        [-0.8, -0.4]  # V_EXIT range
    ])
    
    n_initial_points = 10  # Random points to warm up the model
    n_iterations = 50      # Bayesian Optimization steps
    kappa = 2.5            # Exploration/Exploitation balance (higher = more exploration)

# ==========================================
# 2. INSTRUMENT CONTROL CLASS 🎛️
# ==========================================
class InstrumentController:
    def __init__(self, config):
        self.rm = pyvisa.ResourceManager()
        self.cfg = config
        
        try:
            # Initialize Sources
            self.yoko_ent = self.rm.open_resource(self.cfg.addr_yoko_ent)
            self.yoko_p = self.rm.open_resource(self.cfg.addr_yoko_p)
            self.yoko_exit = self.rm.open_resource(self.cfg.addr_yoko_exit)
            self.dmm = self.rm.open_resource(self.cfg.addr_dmm)
            
            # Basic Setup (Verify specific commands for your models)
            for inst in [self.yoko_ent, self.yoko_p, self.yoko_exit]:
                inst.write_termination = '\n'
                inst.read_termination = '\n'
                # inst.write("F1R5O1E") # Example setup string from your file
                
            print(f"✅ Instruments Connected. Target Current: {self.cfg.target_current*1e12:.2f} pA")
            
        except Exception as e:
            print(f"⚠️ Connection Error (Simulation Mode Active): {e}")
            self.simulation_mode = True
        else:
            self.simulation_mode = False

    def set_voltages(self, v_ent, v_p, v_exit):
        """Applies voltages to the three gates."""
        if not self.simulation_mode:
            # Format: S-0.50000E (Yokogawa style)
            self.yoko_ent.write(f'S{v_ent:.5f}E')
            self.yoko_p.write(f'S{v_p:.5f}E')
            self.yoko_exit.write(f'S{v_exit:.5f}E')
            time.sleep(0.1) # Settling time

    def measure_current(self):
        """Reads current from DMM."""
        if not self.simulation_mode:
            try:
                # Trigger measurement
                val = float(self.dmm.query("READ?"))
                return val
            except:
                return 0.0
        else:
            # 🧪 SIMULATION for testing logic without hardware
            # Simulates a plateau around -0.6V for all gates
            # True current = e*f, we add noise and 'distance' penalty
            global current_x
            v_ent, v_p, v_exit = current_x
            dist = np.sqrt((v_ent + 0.6)**2 + (v_p + 0.6)**2 + (v_exit + 0.6)**2)
            
            # Simulate a plateau (current is close to target if dist is small)
            sim_current = self.cfg.target_current * (1.0 + 0.5 * (1 - np.exp(-10*dist**2)))
            noise = np.random.normal(0, 1e-15)
            return sim_current + noise

# ==========================================
# 3. BAYESIAN OPTIMIZATION LOGIC 🧠
# ==========================================
class BayesianOptimizer:
    def __init__(self, bounds):
        self.bounds = bounds
        # Kernel: Matern or RBF + WhiteKernel for noise handling
        kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=0.1, length_scale_bounds=(1e-2, 1e1)) + \
                 WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1))
        
        self.gp = GaussianProcessRegressor(kernel=kernel, 
                                           alpha=0.0, 
                                           n_restarts_optimizer=5,
                                           normalize_y=True)
        self.X_train = []
        self.y_train = []

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        self.gp.fit(self.X_train, self.y_train)

    def acquisition_function(self, X, kappa=2.5):
        """Lower Confidence Bound (LCB) - We want to MINIMIZE the cost."""
        mu, sigma = self.gp.predict(X, return_std=True)
        return mu - kappa * sigma

    def suggest_next_point(self):
        """Optimizes the acquisition function to find the next query point."""
        
        # 1. Random sampling to find good starting points for the minimizer
        n_random_samples = 2000
        X_random = np.random.uniform(self.bounds[:, 0], self.bounds[:, 1], size=(n_random_samples, 3))
        acq_values = self.acquisition_function(X_random)
        min_idx = np.argmin(acq_values)
        x0 = X_random[min_idx]

        # 2. Local optimization (L-BFGS-B) to refine the point
        res = minimize(lambda x: self.acquisition_function(x.reshape(1, -1)), 
                       x0=x0, 
                       bounds=self.bounds, 
                       method="L-BFGS-B")
        return res.x

# ==========================================
# 4. MAIN LOOP 🚀
# ==========================================
def objective_function(I_meas, config):
    """
    Calculates |delta I_p| (Logarithmic deviation).
    Formula: log10( | <n> - 1 | )
    """
    n_avg = I_meas / config.target_current
    deviation = np.abs(n_avg - 1)
    
    # Avoid log(0) by adding epsilon
    epsilon = 1e-12 
    cost = np.log10(deviation + epsilon)
    return cost, n_avg

def run_experiment():
    cfg = Config()
    instr = InstrumentController(cfg)
    bo = BayesianOptimizer(cfg.bounds)
    
    # Data logging
    X_history = []
    y_history = []
    
    plt.figure(figsize=(10, 6))
    plt.ion() # Interactive mode on
    
    print("\n--- Starting Quantum Plateau Search ---")
    print(f"{'Iter':<5} {'V_ENT':<8} {'V_P':<8} {'V_EXIT':<8} {'Current (pA)':<15} {'<n>':<10} {'Cost':<10}")
    print("-" * 70)

    # --- 1. Initialization Phase (Random Sampling) ---
    for i in range(cfg.n_initial_points):
        # Random point within bounds
        next_x = np.random.uniform(cfg.bounds[:, 0], cfg.bounds[:, 1])
        
        # Hardware interaction
        global current_x # For simulation logic
        current_x = next_x
        instr.set_voltages(*next_x)
        current = instr.measure_current()
        
        # Calculate Cost
        cost, n_val = objective_function(current, cfg)
        
        # Store
        X_history.append(next_x)
        y_history.append(cost)
        
        print(f"{i:<5} {next_x[0]:<8.4f} {next_x[1]:<8.4f} {next_x[2]:<8.4f} {current*1e12:<15.4f} {n_val:<10.4f} {cost:<10.2f}")

    # --- 2. Bayesian Optimization Phase ---
    for i in range(cfg.n_iterations):
        # Train GP
        bo.fit(X_history, y_history)
        
        # Ask for next best point
        next_x = bo.suggest_next_point()
        
        # Measurement
        current_x = next_x
        instr.set_voltages(*next_x)
        current = instr.measure_current()
        
        # Calculate Cost
        cost, n_val = objective_function(current, cfg)
        
        # Store
        X_history.append(next_x)
        y_history.append(cost)
        
        print(f"{i+cfg.n_initial_points:<5} {next_x[0]:<8.4f} {next_x[1]:<8.4f} {next_x[2]:<8.4f} {current*1e12:<15.4f} {n_val:<10.4f} {cost:<10.2f}")
        
        # Live Plotting
        if i % 2 == 0:
            plt.clf()
            plt.plot(y_history, 'o-', color='teal', label='Cost ($\log_{10}|\delta I_p|$)')
            plt.xlabel('Iteration')
            plt.ylabel('Cost (Lower is better)')
            plt.title(f'Bayesian Optimization Progress (Iter {i})')
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.pause(0.1)

    # --- End ---
    best_idx = np.argmin(y_history)
    best_params = X_history[best_idx]
    best_cost = y_history[best_idx]
    
    print("\n🎉 Optimization Complete!")
    print(f"Best Parameters Found:")
    print(f"V_ENT : {best_params[0]:.5f} V")
    print(f"V_P   : {best_params[1]:.5f} V")
    print(f"V_EXIT: {best_params[2]:.5f} V")
    print(f"Min Cost: {best_cost:.4f}")
    
    plt.ioff()
    plt.show()

if __name__ == "__main__":
    run_experiment()